In [1]:
import numpy as np
import time

def my_range(start, end):
    if start <= end:
        return range(start, end + 1)
    else:
        return range(start, end - 1, -1)


class Problem:
    char_mapping = ('·', 'Q')

    def __init__(self, n=4):
        self.n = n

    def is_valid(self, state):
        """
        check the state satisfy condition or not.
        :param state: list of points (in (row id, col id) tuple form)
        :return: bool value of valid or not
        """
        board = self.get_board(state)
        res = True
        for point in state:
            i, j = point
            condition1 = board[:, j].sum() <= 1
            condition2 = board[i, :].sum() <= 1
            condition3 = self.pos_slant_condition(board, point)
            condition4 = self.neg_slant_condition(board, point)
            res = res and condition1 and condition2 and condition3 and condition4
            if not res:
                break
        return res

    def is_satisfy(self, state):
        return self.is_valid(state) and len(state) == self.n

    def pos_slant_condition(self, board, point):
        i, j = point
        tmp = min(self.n - i - 1, j)
        start = (i + tmp, j - tmp)
        tmp = min(i, self.n - j - 1)
        end = (i - tmp,  j + tmp)
        rows = my_range(start[0], end[0])
        cols = my_range(start[1], end[1])
        return board[rows, cols].sum() <= 1

    def neg_slant_condition(self, board, point):
        i, j = point
        tmp = min(i, j)
        start = (i - tmp, j - tmp)
        tmp = min(self.n - i - 1, self.n - j - 1)
        end = (i + tmp, j + tmp)
        rows = my_range(start[0], end[0])
        cols = my_range(start[1], end[1])
        return board[rows, cols].sum() <= 1

    def get_board(self, state):
        board = np.zeros([self.n, self.n], dtype=int)
        for point in state:
            board[point] = 1
        return board

    def print_state(self, state):
        board = self.get_board(state)
        print('_' * (2 * self.n + 1))
        for row in board:
            for item in row:
                print(f'|{Problem.char_mapping[item]}', end='')
            print('|')
        print('-' * (2 * self.n + 1))



# BTS: Backtracking search (Request 1, DDL: 3.31)

In [2]:
from numba import njit

def bts(problem: Problem):
    action_stack = []
    row = 0
    col = 0
    # while not problem.is_satisfy(action_stack):
    try:
        while len(action_stack) < problem.n:
            action_stack.append((row, col))
            if problem.is_valid(action_stack):
                row += 1
                col = 0
                continue
            else:
                action_stack.remove((row, col))
                col += 1
            
            while col == problem.n:
                row -= 1

                col = action_stack.pop()[1] + 1
    except:
        return []
    return action_stack


# Improving BTS To DO (Request 2, DDL: 4.07)
* Which variable should be assigned next?
* In what order should its values be tried?
* Can we detect inevitable failure early?

### Minimum Remaining Value
Choose the variable with the fewest legal values in its domain
### Least constraining value
Given a variable, choose the least constraining value: the one that rules out the fewest values in the remaining variables
### Forward checking
Keep track of remaining legal values for the unassigned variables. Terminate when any variable has no legal values.

In [3]:
N = 16

true_board = np.ones((N, N), dtype=bool)
false_board = np.zeros((N, N), dtype=bool)
t, f = True, False

from numpy import copy
def get_new_valid_board(valid_board, move):
    
    new_board = copy(valid_board)
    new_board[:, move[1]] = new_board[move[0], :] = False

    i, j = move
    tmp = min(N - i - 1, j)
    start = (i + tmp, j - tmp)
    tmp = min(i, N - j - 1)
    end = (i - tmp,  j + tmp)
    rows = my_range(start[0], end[0])
    cols = my_range(start[1], end[1])
    new_board[rows, cols] = False

    tmp = min(i, j)
    start = (i - tmp, j - tmp)
    tmp = min(N - i - 1, N - j - 1)
    end = (i + tmp, j + tmp)
    rows = my_range(start[0], end[0])
    cols = my_range(start[1], end[1])
    new_board[rows, cols] = False
    
    return new_board


pos_set = [(i, j) for i in range(N) for j in range(N)]
def improving_bts(problem: Problem):
    action_stack = []
    valid_boards = []
    row = 0
    col = 0
    valid_board = np.copy(true_board)

    while len(action_stack) < N:
        while col == N:
            row -= 1
            col = action_stack.pop()[1] + 1
            valid_boards.pop()
            if len(valid_boards) == 0:
                valid_board = true_board
            else:
                valid_board = valid_boards[-1]

        # 某一行没有了
        if (valid_board.sum(axis=1) == 0).sum() > row:
            col = N
            continue


        move = pos_set[row * N + col]
        if not valid_board[move]:
            col += 1
            continue

        
        new_valid_board = get_new_valid_board(valid_board, move)
        if new_valid_board.sum() == 0 and len(action_stack) != N - 1:
            col += 1
            continue


        action_stack.append(move)
        valid_boards.append(new_valid_board)
        valid_board = new_valid_board
        row += 1
        col = 0

    return action_stack



In [4]:
def test():
    # test_block
    n = N # Do not modify this parameter, if you want to change the size, go to the first line of whole program.
    render = False # here to select GUI or not
    method = improving_bts  # here to change your method; bts or improving_bts
    p = Problem(n)
    if render:
        import pygame
        w, h = 90 * n + 10, 90 * n + 10
        screen = pygame.display.set_mode((w, h))
        screen.fill('white')
        actions = method(p)
        idx = 0
        clk = pygame.time.Clock()
        queen_img = pygame.image.load('./queen.png')
        while True:
            for event in pygame.event.get():
                if event == pygame.QUIT:
                    exit()
            try:
                actions = actions[idx]
                idx += 1
                screen.fill('white')
                for i in range(n + 1):
                    pygame.draw.rect(screen, 'black', (i * 90, 0, 10, h))
                    pygame.draw.rect(screen, 'black', (0, i * 90, w, 10))
                for action in actions:
                    i, j = action
                    screen.blit(queen_img, (10 + 90 * j, 10 + 90 * i))
                pygame.display.flip()
            except:
                pass
            clk.tick(5)
        pass
    else:
        start_time = time.time()
        actions = method(p)
        print(actions)
        p.print_state(actions)
        print(time.time() - start_time)


import cProfile
cProfile.run("test()")

[(0, 0), (1, 2), (2, 4), (3, 1), (4, 12), (5, 8), (6, 13), (7, 11), (8, 14), (9, 5), (10, 15), (11, 6), (12, 3), (13, 10), (14, 7), (15, 9)]
_________________________________
|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|
|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|
|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|
|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|
|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|
|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|
|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|
|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|
---------------------------------
1.2519536018371582
         682342 function calls in 1.252 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    1.252    1.252 1112748918.py:1(test)
        1